In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv("edu_enrollees.csv")
df.head(2)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train


In [3]:
df = df.drop(columns = ["city", "company_size", "company_type"])

In [5]:
df_na_cnt = pd.DataFrame(dict(types = df.dtypes,
                              na_cnt = df.isna().sum()))
df_na_cnt

,types,na_cnt
enrollee_id,float64,0
city_development_index,float64,0
gender,object,4508
relevant_experience,object,0
enrolled_university,object,386
education_level,object,460
major_discipline,object,2813
experience,object,65
last_new_job,object,423
training_hours,float64,0


In [6]:
df = df.dropna()

In [7]:
df = df.loc[~df["experience"].isin([">20", "<1"]), ]
df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]
df["experience"] = df["experience"].astype("int")
df["last_new_job"] = df["last_new_job"].astype("int")

In [8]:
len(df)

7522

In [9]:
df_base = df.reset_index(drop = True)

### Q1.

In [10]:
df_q1 = df_base[["relevant_experience", "target"]].copy()
df_q1.head(2)

,relevant_experience,target
0,Has relevant experience,1.0
1,Has relevant experience,0.0


In [11]:
df_A = df_q1.loc[df_q1["relevant_experience"] == "No relevant experience", ]
df_B = df_q1.loc[df_q1["relevant_experience"] == "Has relevant experience", ]

In [15]:
stat_A = df_A["target"].value_counts(normalize = True)[1]
stat_B = df_B["target"].value_counts(normalize = True)[1]
stat_A, stat_B

(0.38287331917905165, 0.21591095105581928)

In [17]:
round(stat_A / stat_B, 2)

1.77

In [18]:
df_base.groupby("relevant_experience")["target"].mean()

relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64

### Q2.

In [25]:
df_q2_obj = df_base.loc[:, "gender":"major_discipline"]
df_q2_dum = pd.get_dummies(df_q2_obj, columns = df_q2_obj.columns, dtype = "int")
df_q2_dum.head(1)

,gender_Female,gender_Male,gender_Other,relevant_experience_Has relevant experience,relevant_experience_No relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1


In [26]:
df_q2_dum.columns[[2, 4]]

Index(['gender_Other', 'relevant_experience_No relevant experience'], dtype='object')

In [31]:
df_q2_dum = df_q2_dum.drop(columns = df_q2_dum.columns[[2, 4, 7, 10, 16]])
df_q2_dum.head(1)

,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,0,1,1,0,0,1,0,0,0,0,0,0


In [30]:
# df_q2_dum.head(1).transpose().reset_index()

In [33]:
col_1 = ["target", "Xgrp"]
col_2 = ["city_development_index", "experience", "last_new_job", "training_hours"]
df_job2 = pd.concat([df_base[col_1 + col_2], df_q2_dum], axis = 1)
df_job2.head(2)

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,1.0,train,0.92,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
1,0.0,train,0.92,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0


In [35]:
df_job2.isna().sum().sum()

0

In [38]:
model_lr = LogisticRegression(C = 100000, max_iter = 1000, solver = "liblinear", 
                              fit_intercept = True,
                              random_state = 123) 
model_lr.fit(X = df_job2.drop(columns = ["target", "Xgrp"]), 
             y = df_job2["target"])
model_lr.coef_ # 회귀계수

array([[-6.11732384e+00, -2.85015530e-02,  9.56531984e-02,
        -9.26206002e-04, -1.62975865e-01, -1.36723955e-01,
        -7.60567385e-01,  5.14109835e-01, -2.81496522e-01,
         3.23116412e-01,  1.67418791e-02,  2.61665001e-01,
         1.07607195e-01,  2.43479152e-01,  3.97900227e-01,
        -4.39387752e-01]])

In [52]:
model_lr.intercept_ # <-- 고려 XXX

array([4.36024692])

In [53]:
pd.DataFrame(model_lr.coef_, columns = df_job2.columns[2:])

,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,-6.117324,-0.028502,0.095653,-0.000926,-0.162976,-0.136724,-0.760567,0.51411,-0.281497,0.323116,0.016742,0.261665,0.107607,0.243479,0.3979,-0.439388


In [42]:
round(np.exp(model_lr.coef_).max(), 4) # 1.67

1.6721

In [43]:
np.floor(np.exp(model_lr.coef_).max() * 100) / 100

1.67

### Q3.

In [54]:
df_job2["Xgrp"].unique()

array(['train', 'test'], dtype=object)

In [55]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
df_test  = df_job2.loc[df_job2["Xgrp"] == "test",  ]
len(df_train), len(df_test)

(4706, 2816)

In [57]:
model_knn = KNeighborsClassifier(n_neighbors = 5)
model_knn.fit(X = df_train.drop(columns = ["target", "Xgrp"]),
              y = df_train["target"])
pred = model_knn.predict(df_test.drop(columns = ["target", "Xgrp"]))
pred[:4]

array([0., 0., 0., 0.])

In [58]:
y_t = df_test["target"]
y_p = pred

In [67]:
df_tab = pd.crosstab(y_t, y_p)
df_tab.values

array([[1899,  193],
       [ 616,  108]], dtype=int64)

In [69]:
round(df_tab.values.diagonal().sum() / df_tab.values.sum(), 2)

0.71

In [70]:
from sklearn.metrics import accuracy_score

In [71]:
round(accuracy_score(y_true = y_t, y_pred = y_p), 2)

0.71

In [72]:
import sklearn.metrics as sm

In [76]:
# dir(sm)
[m for m in dir(sm) if m[-5:] == "error"]

['coverage_error',
 'max_error',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'median_absolute_error']

In [78]:
# dir(df_tab)